In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('labour.csv')

 - Here I am aiming to save my dataset in mysql database and use it in my project. I will use the sqlalchemy library for this.
 - I will have to define an engine() for this process, but before that, two different situations arise as a connection string. This is because I will first connect to my database and create a database with a name I want. Then, in order to save my dataset to the database, I need to reconnect my engine() connection with the name of the database I created.
 - It looks like a code mess, but whatever method I tried, I couldn't succeed. For the reasons I explained in the previous article, I had to manage engine() with two different connection strings in this way.
 - Instead of using a local mysql solution, I created an azure mysql database, which is one of the free services specially defined for students, using the azure portal of the microsoft company and I will use it in my project.

In [5]:
from sqlalchemy import create_engine, text

user = 'ca2023'
password = 'CCT2023!'
host = 'cctmysql.mysql.database.azure.com'
port = 3306
database = 'cct_db'

engine_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}'
db_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(engine_url, echo=True)

with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database};"))
    conn.close()

    

2023-05-13 16:15:51,545 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-13 16:15:51,547 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:52,049 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-13 16:15:52,050 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:52,312 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-13 16:15:52,313 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:52,630 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-13 16:15:52,630 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS cct_db;
2023-05-13 16:15:52,631 INFO sqlalchemy.engine.Engine [generated in 0.16460s] {}
2023-05-13 16:15:52,769 INFO sqlalchemy.engine.Engine ROLLBACK


 - Now that I have created my database, I will send the dataframe to my database using my connection string containing the database for engine().
 - I carry out my process by checking for errors due to the possibility of the current table name being in the database.
 - I am performing my operation using the **to_sql()** function of the pandas library.

In [6]:
engine = create_engine(db_url, echo=True)
try:
    df.to_sql('labour', con=engine, if_exists='fail', index=False)
except Exception as ex:
    print(ex)
else:
    print("Table created")

2023-05-13 16:15:53,847 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-13 16:15:53,848 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:54,366 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-13 16:15:54,368 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:54,625 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-13 16:15:54,626 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:55,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-13 16:15:55,091 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-13 16:15:55,091 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:55,175 INFO sqlalchemy.engine.Engine ROLLBACK
Table 'labour' already exists.


In [7]:
conn = engine.connect()
sqldf = pd.read_sql_table('labour', conn)

2023-05-13 16:15:55,452 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-13 16:15:55,453 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-13 16:15:55,453 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:55,724 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `labour`
2023-05-13 16:15:55,725 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-13 16:15:55,989 INFO sqlalchemy.engine.Engine SELECT labour.`STRUCTURE`, labour.`STRUCTURE_ID`, labour.freq, labour.indic_bt, labour.nace_r2, labour.s_adj, labour.unit, labour.geo, labour.`TIME_PERIOD`, labour.`OBS_VALUE`, labour.`OBS_FLAG` 
FROM labour
2023-05-13 16:15:55,990 INFO sqlalchemy.engine.Engine [generated in 0.17848s] {}


In [8]:
sqldf.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2005,76.9,None
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2006,76.7,None
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2007,74.1,None
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2008,97.3,None
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2009,103.7,None


In [9]:
sqldf.shape

(9314, 11)

In [10]:
sqldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9314 entries, 0 to 9313
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STRUCTURE     9314 non-null   object 
 1   STRUCTURE_ID  9314 non-null   object 
 2   freq          9314 non-null   object 
 3   indic_bt      9314 non-null   object 
 4   nace_r2       9314 non-null   object 
 5   s_adj         9314 non-null   object 
 6   unit          9314 non-null   object 
 7   geo           9314 non-null   object 
 8   TIME_PERIOD   9314 non-null   int64  
 9   OBS_VALUE     9108 non-null   float64
 10  OBS_FLAG      1717 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 800.6+ KB


<table border="1" cellspacing="0" cellpadding="0">
    <tbody>
        <tr>
            <td width="301" valign="top">
                <p>
                    Column Name
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Description
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    freq
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Frequency of publication of data
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    Indic_bt
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    The type of statistic (employment, production, sales, etc.)
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    nace_r2
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Code showing which industry classification the data relates
                    to
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    s_adj
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Code indicating whether it is seasonally adjusted or
                    unadjusted
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    unit
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    unit of measurement
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    geo
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Country
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    TIME_PERIOD
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Year
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    OBS_VALUE
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    Value of data <br/>
                    <br/>
                </p>
            </td>
        </tr>
        <tr>
            <td width="301" valign="top">
                <p>
                    OBS_FLAG
                </p>
            </td>
            <td width="301" valign="top">
                <p>
                    A code that provides additional information about the
                    accuracy or reliability of the data
                </p>
            </td>
        </tr>
    </tbody>
</table>

In [11]:
sqldf.describe()

,TIME_PERIOD,OBS_VALUE
count,9314.000000,9108.000000
mean,2010.120893,72.166359
std,6.580738,54.231636
min,1990.000000,-67.400000
25%,2005.000000,8.100000
50%,2010.000000,91.200000
75%,2015.000000,106.000000
max,2022.000000,406.500000


In [12]:
sqldf.describe(include="object")

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,OBS_FLAG
count,9314,9314,9314,9314,9314,9314,9314,9314,1717
unique,1,1,1,3,1,2,3,36,5
top,dataflow,ESTAT:STS_COLB_A(1.0),A,WAGE,F,NSA,I15,NL,p
freq,9314,9314,9314,3546,9314,5728,3931,338,866


- I will use the uniqe() function to display what are the unique values ​​in my categorical columns. 
- I will also use tabulate, a simple library of tables, to display values ​​neatly

In [13]:
from tabulate import tabulate

table_data = {
    "indic_bt": sqldf.indic_bt.unique(),
    "s_adj": sqldf.s_adj.unique(),
    "unit": sqldf.unit.unique()
}

table = tabulate(table_data, headers="keys", tablefmt="psql")

print(table)

+------------+---------+--------+
| indic_bt   | s_adj   | unit   |
|------------+---------+--------|
| EMPL       | NSA     | I10    |
| HOWK       | CA      | I15    |
| WAGE       |         | PCH_SM |
+------------+---------+--------+


### I've added a small table explanation to explain what the values ​​for my categorical columns mean, the abbreviation values ​​on the eurostat website
<table border="1" cellspacing="0" cellpadding="0">
    <tbody>
        <tr>
            <td width="204" colspan="2" valign="top">
                <p align="center">
                    Indic_bt
                </p>
            </td>
            <td width="198" colspan="2" valign="top">
                <p align="center">
                    S_adj
                </p>
            </td>
            <td width="199" colspan="2" valign="top">
                <p align="center">
                    unit
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    EMPL
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Employment (number of persons employed)
                </p>
            </td>
            <td width="42" valign="top">
                <p>
                    NSA
                </p>
            </td>
            <td width="156" valign="top">
                <p>
                    Unadjusted data (i.e. neither seasonally adjusted nor
                    calendar adjusted data)
                </p>
            </td>
            <td width="70" valign="top">
                <p>
                    I10
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Index, 2010=100
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    HOWK
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Volume of work done (hours worked)
                </p>
            </td>
            <td width="42" valign="top">
                <p>
                    CA
                </p>
            </td>
            <td width="156" valign="top">
                <p>
                    Calencar adjusted data, not seasonally adjusted data
                </p>
            </td>
            <td width="70" valign="top">
                <p>
                    I15
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Index, 2015=100
                </p>
            </td>
        </tr>
        <tr>
            <td width="58" valign="top">
                <p>
                    WAGE
                </p>
            </td>
            <td width="146" valign="top">
                <p>
                    Gross wages and salaries
                </p>
            </td>
            <td width="42" valign="top">
            </td>
            <td width="156" valign="top">
            </td>
            <td width="70" valign="top">
                <p>
                    PCH_SM
                </p>
            </td>
            <td width="129" valign="top">
                <p>
                    Percentage change compared to same period in previous year
                </p>
            </td>
        </tr>
    </tbody>
</table>

- Since I will be using the employment data, which is the main subject of my project, I will take the EMPL values in the indic_bt column. 
- Preferably by using the PCH_SM values from the unit column

In [14]:
filtered_df = sqldf.loc[(df['indic_bt'] == 'EMPL') & (df['unit'] == 'PCH_SM')]

In [15]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
1494,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2006,-0.3,None
1495,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2007,-3.3,None
1496,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2008,31.4,None
1497,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2009,6.5,None
1498,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2010,-3.6,None


In [16]:
filtered_df.shape

(803, 11)

In [17]:
filtered_df.describe(include="object")

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,OBS_FLAG
count,803,803,803,803,803,803,803,803,70
unique,1,1,1,1,1,1,1,36,4
top,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,FR,e
freq,803,803,803,803,803,803,803,32,34


When I look at the records in the first 5 rows of my filtered dataset, I notice that my index values are not normal. Since I have created a new dataframe by filtering from my existing dataset according to certain criteria, the index values appear as row numbers in my current dataset. I will use the reset_index() function to fix this and apply sequential index values to the dataframe.

In [18]:
filtered_df.reset_index(drop=True, inplace=True)

In [19]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2006,-0.3,None
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2007,-3.3,None
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2008,31.4,None
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2009,6.5,None
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2010,-3.6,None


Now that I have done the necessary filtering according to the categorical columns of my dataset, I can now start my operations with numeric columns. For this, first of all, I want to check if there are any empty or missing values in my rows.

In [20]:
# Check for missing values
filtered_df.isnull().sum()

STRUCTURE         0
STRUCTURE_ID      0
freq              0
indic_bt          0
nace_r2           0
s_adj             0
unit              0
geo               0
TIME_PERIOD       0
OBS_VALUE        15
OBS_FLAG        733
dtype: int64

I want to select the years in which the obs_value column of all countries is not null in all years

In [21]:
non_null_counts = filtered_df.groupby('geo')['OBS_VALUE'].count()
non_null_counts

geo
AL    17
AT    26
BE    21
BG    22
CH    24
CY    22
CZ    22
DE    22
DK    14
EE    24
EL    22
ES    22
FI    22
FR    32
HR    22
HU    22
IE    22
IS    14
IT    22
LT    24
LU    22
LV    22
ME    22
MK    15
MT    22
NL    26
NO    22
PL    22
PT    22
RO    22
RS    22
SE    22
SI    22
SK    27
TR    17
UK    23
Name: OBS_VALUE, dtype: int64

In [22]:
# Finding the number of filled values ​​for each year
non_null_years = filtered_df.groupby('TIME_PERIOD')['OBS_VALUE'].count()

# Select only full years
non_null_years = non_null_years[non_null_years == len(filtered_df['geo'].unique())]


In [23]:
# Filtering dataset by full years
filtered_df = filtered_df[filtered_df['TIME_PERIOD'].isin(non_null_years.index)]


In [24]:
# Calculating the number of full years for each country
non_null_counts = filtered_df.groupby('geo')['OBS_VALUE'].count()
non_null_counts

geo
AL    11
AT    11
BE    11
BG    11
CH    11
CY    11
CZ    11
DE    11
DK    11
EE    11
EL    11
ES    11
FI    11
FR    11
HR    11
HU    11
IE    11
IS    11
IT    11
LT    11
LU    11
LV    11
ME    11
MK    11
MT    11
NL    11
NO    11
PL    11
PT    11
RO    11
RS    11
SE    11
SI    11
SK    11
TR    11
UK    11
Name: OBS_VALUE, dtype: int64

In [25]:
print(filtered_df["TIME_PERIOD"].unique())

[2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]


In [26]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 3 to 802
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STRUCTURE     396 non-null    object 
 1   STRUCTURE_ID  396 non-null    object 
 2   freq          396 non-null    object 
 3   indic_bt      396 non-null    object 
 4   nace_r2       396 non-null    object 
 5   s_adj         396 non-null    object 
 6   unit          396 non-null    object 
 7   geo           396 non-null    object 
 8   TIME_PERIOD   396 non-null    int64  
 9   OBS_VALUE     396 non-null    float64
 10  OBS_FLAG      15 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 37.1+ KB


In [27]:
filtered_df.isnull().sum()

STRUCTURE         0
STRUCTURE_ID      0
freq              0
indic_bt          0
nace_r2           0
s_adj             0
unit              0
geo               0
TIME_PERIOD       0
OBS_VALUE         0
OBS_FLAG        381
dtype: int64

In my dataset, you can see that the geo column, that is, the column representing the countries, consists of only a 2-digit country code. Now I want to add a new column named country to my dataset and I want to add the names of the countries according to the country codes in the geo column to this country column.

In [28]:
import pycountry

def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_2=code).name
    except:
        return None

In [29]:
filtered_df['country'] = filtered_df['geo'].apply(get_country_name)

In the method, I specify that my country codes are 2 digits with alpha2. If it had 3 digits I would use alpha3. If it can't find any country code in the method, I wanted it to add the value None as a check. Now according to this situation, do I have a value of None or I want to see if I have successfully converted all country codes?

In [30]:
filtered_df.sample(5)

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
507,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,ME,2012,3.8,None,Montenegro
766,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,TR,2009,-67.4,None,Turkey
195,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,DK,2014,2.7,None,Denmark
461,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,LU,2010,-0.6,None,Luxembourg
750,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,SK,2010,-2.6,None,Slovakia


Yes, exactly as I thought. When I list the example 5 rows in my dataset, I observe that the value of the country with the EL code is None. Now, for this, it is necessary to filter the None values ​​of the country names in the country column and produce a solution for this.

In [31]:
missing_countries = filtered_df[filtered_df['country'].isnull()]['geo'].unique()
missing_countries

array(['EL', 'UK'], dtype=object)

I can see now that I couldn't find the names of the countries with EL and UK codes in my library. When I search these country codes on the internet, I can see that the EL code corresponds to Greece and the UK code to United Kingdom.
Now I will look at the corresponding country codes in my library according to the names of these countries and replace the EL and UK values in the geo column in my own dataset.


In [32]:
print(pycountry.countries.get(name='Greece').alpha_2)

GR


In [33]:
print(pycountry.countries.get(name='United Kingdom').alpha_2)

GB


In [34]:
filtered_df['geo'] = filtered_df['geo'].replace({'EL': 'GR', 'UK': 'GB'})

In [35]:
filtered_df['country'] = filtered_df['geo'].apply(get_country_name)

In [36]:
filtered_df.sample(5)

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
798,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2015,3.7,None,United Kingdom
664,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,RO,2017,-0.9,None,Romania
592,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,NO,2011,3.8,None,Norway
101,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,CH,2013,1.4,e,Switzerland
792,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2009,-3.1,None,United Kingdom


- Now all of my country codes have been completed according to the ISO3166 standard. Also, None values are left in my country names column. let's check it again.
- Yes now I can tell from an empty string result that I don't have a line with the value None in the country names column.

In [37]:
missing_countries = filtered_df[filtered_df['country'].isnull()]['geo'].unique()
missing_countries

array([], dtype=object)

In [38]:
filtered_df.describe()

,TIME_PERIOD,OBS_VALUE
count,396.000000,396.000000
mean,2014.000000,-0.490657
std,3.166278,8.890347
min,2009.000000,-67.400000
25%,2011.000000,-3.400000
50%,2014.000000,0.600000
75%,2017.000000,3.800000
max,2019.000000,27.900000


In [39]:
filtered_df.describe(include="object")

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,OBS_FLAG,country
count,396,396,396,396,396,396,396,396,15,396
unique,1,1,1,1,1,1,1,36,2,36
top,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,e,Albania
freq,396,396,396,396,396,396,396,11,13,11


In [40]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 3 to 802
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STRUCTURE     396 non-null    object 
 1   STRUCTURE_ID  396 non-null    object 
 2   freq          396 non-null    object 
 3   indic_bt      396 non-null    object 
 4   nace_r2       396 non-null    object 
 5   s_adj         396 non-null    object 
 6   unit          396 non-null    object 
 7   geo           396 non-null    object 
 8   TIME_PERIOD   396 non-null    int64  
 9   OBS_VALUE     396 non-null    float64
 10  OBS_FLAG      15 non-null     object 
 11  country       396 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 40.2+ KB


In [41]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2009,6.5,None,Albania
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2010,-3.6,None,Albania
5,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2011,-8.8,None,Albania
6,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2012,3.2,None,Albania
7,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2013,-5.8,None,Albania


In [42]:
filtered_df.tail()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
798,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2015,3.7,None,United Kingdom
799,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2016,2.7,None,United Kingdom
800,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2017,5.6,None,United Kingdom
801,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2018,0.3,None,United Kingdom
802,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,GB,2019,2.9,None,United Kingdom


I had reset my indexes before, but after that process, I applied to filter operations again, so I see that my indexes are not in order. So I need to reset my indexes again

In [43]:
# reset index
filtered_df.reset_index(drop=True, inplace=True)

In [44]:
filtered_df.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2009,6.5,None,Albania
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2010,-3.6,None,Albania
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2011,-8.8,None,Albania
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2012,3.2,None,Albania
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,AL,2013,-5.8,None,Albania


In [45]:
import plotly.express as px

fig = px.choropleth(filtered_df, 
                    locations="country",
                    locationmode="country names",
                    color="OBS_VALUE", 
                    hover_name="country", 
                    # column to add to hover information
                    animation_frame="TIME_PERIOD", 
                    # column on which to animate
                    color_continuous_scale=px.colors.sequential.Plasma)
                    
fig.update_layout(
    # add a title text for the plot
    title_text = 'Labour in Europe',
    # set projection style for the plot
    geo = dict(projection={'type':'natural earth'}) # by default, projection type is set to 'equirectangular'
)
fig.show()

In [46]:
# line chart
fig = px.line(
    filtered_df,
    x="TIME_PERIOD",
    y="OBS_VALUE",
    color='country',
    labels={'TIME_PERIOD': 'Year', 'OBS_VALUE': 'Value', 'country': 'Country'}, # set the labels for the chart
    title='Labour in Europe',
    hover_name='country',
    
)
fig.show()

- I have performed the necessary visualization processes of my existing data set and now I want to start comparing Ireland with different countries.
- I will use Ireland and the United Kingdom in my research.


In [47]:
df_ire_uk = filtered_df[filtered_df['country'].isin(['Ireland', 'United Kingdom'])]


In [49]:
fig = px.bar(df_ire_uk, x='TIME_PERIOD', y='OBS_VALUE', color='country', barmode='group')
fig.show()

In [50]:
df_ire_uk

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,country
176,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2009,-31.6,None,Ireland
177,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2010,-28.6,None,Ireland
178,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2011,-7.9,None,Ireland
179,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2012,-7.4,None,Ireland
180,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2013,0.4,None,Ireland
181,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2014,11.7,None,Ireland
182,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2015,17.4,None,Ireland
183,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2016,14.2,None,Ireland
184,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2017,8.2,None,Ireland
185,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,PCH_SM,IE,2018,12.1,None,Ireland
